In [7]:
pip install -U langchain-openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 KB 127.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 KB 126.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sarvamai import SarvamAI
SARVAM_API_KEY = "0140c592-6272-4436-8398-111390895f78"
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

# PlanSearch generator
def generate_setups(topic, n=5):
    prompt = f"""Generate {n} unique and original joke setups about the topic "{topic}".
Only return the setups, numbered, with no punchlines or explanations. Example format:

1. Why did the penguin wear sunglasses at night?
2. What happens when a penguin tries online dating?

Now generate {n} setups in that format."""
    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=300
    )
    
    setups_raw = response.choices[0].message.content
    
    # Parse list format (1. ..., 2. ..., etc.)
    setups = [line.strip("1234567890. ").strip() for line in setups_raw.split("\n") if line.strip()]
    return setups[:n]

def generate_punchlines(setup, n=3):
    prompt = f"""Generate {n} punchlines for the following joke setup:

"{setup}"

Only return the punchlines, numbered, with no extra commentary or explanation. Example:

1. Because he was trying to look cool in the moonlight.
2. Because the club only played Arctic Monkeys.

Now generate {n} punchlines in that format."""

    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=300
    )
    
    punchlines_raw = response.choices[0].message.content
    
    punchlines = [line.strip("1234567890. ").strip() for line in punchlines_raw.split("\n") if line.strip()]
    return punchlines[:n]

def generate_jokes(topic="penguins", n_setups=3, n_punchlines=2):
    setups = generate_setups(topic, n=n_setups)
    jokes = []

    for setup in setups:
        punchlines = generate_punchlines(setup, n=n_punchlines)
        for punch in punchlines:
            jokes.append(f"{setup} {punch}")

    return jokes

joke_list = []
jokes = generate_jokes("penguins", n_setups=3, n_punchlines=2)
for i, joke in enumerate(jokes, 1):
    print(f"{i}. {joke}")
    joke_list.append(joke)

1. Why did the penguin bring a ladder to the bar? Because he heard the drinks were on the house
2. Why did the penguin bring a ladder to the bar? Because he wanted to reach the high-smС
3. What do you call a penguin that love the trapeze? A penguin with a death wish
4. What do you call a penguin that love the trapeze? A frozen tough of the track
5. How does a penguin build its house? He starts with a foundation of ice cubes
6. How does a penguin build its house? He follows a do-it-yourselfkit from IKEA called "Frosty's Buddy Nest."


In [4]:
def query_openrouter(model, messages):
    body = {
        "model": model,
        "messages": messages,
        "temperature": 0.7
    }
    response = requests.post(BASE_URL, headers=HEADERS, json=body)
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return "[ERROR]"
    return response.json()["choices"][0]["message"]["content"]

import re

def extract_final_score(verdict_text):
    match = re.search(r'(\d+)\s+out of\s+10', verdict_text)
    if match:
        return int(match.group(1))
    else:
        return None

In [6]:
import requests
import json
import time

# Set up base config
API_KEY = "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"  # OpenRouter API key
BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Define the joke
#joke = "Why did the penguin bring sunscreen? Because he was going to a chill party."

score_list = []
score_and_joke_dict = {}
score_and_joke_list = []

for joke in joke_list:

    # Judges with different LLMs
    judges = [
        {
            "name": "Judge A",
            "description": "A sarcastic and cynical comic who despises puns.",
            "model": "mistralai/mixtral-8x7b-instruct"
        },
        {
            "name": "Judge B",
            "description": "A surrealist poet who celebrates absurd and chaotic humor.",
            "model": "meta-llama/llama-3-8b-instruct"
        },
        {
            "name": "Judge C",
            "description": "A professional comedy reviewer with 20 years of balanced experience.",
            "model": "anthropic/claude-3-haiku"
        }
    ]

    # Initial system prompt and setup
    conversation = [
        {"role": "system", "content": "You are a panel of comedy judges discussing and evaluating a joke."},
        {"role": "user", "content": f"The joke is:\n\"{joke}\""},
        {"role": "user", "content": "Each judge will rate the joke (1-10), justify their rating in character, and discuss with others."}
    ]

    # Simulate each judge's initial response
    for judge in judges:
        print(f"\n🔹 {judge['name']} is responding...")
        messages = conversation + [
            {"role": "system", "content": f"You are {judge['name']}. {judge['description']}"},
            {"role": "user", "content": f"As {judge['name']}, give your rating and reasoning for the joke."}
        ]
        reply = query_openrouter(judge["model"], messages)
        conversation.append({"role": "assistant", "content": f"{judge['name']}: {reply}"})
        print(f"{judge['name']}:\n{reply}\n{'-'*60}")
        time.sleep(2)  # gentle pause to avoid rate limits

    # Final verdict from Judge C
    print("\n🎯 === FINAL VERDICT ===\n")
    final_prompt = conversation + [
        {
            "role": "system",
            "content": "You are Judge C. Based on the discussion, summarize the panel's opinions and give a final score with reasoning. Final Score:<number>"
        },
        {
            "role": "user",
            "content": "Judge C, please give your final verdict based on the discussion so far."
        }
    ]

    verdict = query_openrouter("anthropic/claude-3-haiku", final_prompt)
    rating = extract_final_score(verdict)
    score_list.append(rating)
    score_and_joke_dict = {
        "joke":joke,
        "score":rating
    }
    score_and_joke_list.append(score_and_joke_dict)
    print(verdict)


🔹 Judge A is responding...
Judge A:
 Judge A: (sighs) Alright, I'll go first. I give this joke a 2 out of 10. I have to admit, it's not entirely devoid of humor, but as a pun, it falls flat for me. I've never been a fan of such wordplay, no matter how clever or original it might be. I guess this joke just isn't my cup of tea. I appreciate the effort, but it's hard for me to look past the groan-worthy nature of the punchline.
------------------------------------------------------------

🔹 Judge B is responding...
Judge B:
Judge A:  Judge A: (sighs) Alright, I'll go first. I give this joke a 2 out of 10. I have to admit, it's not entirely devoid of humor, but as a pun, it falls flat for me. I've never been a fan of such wordplay, no matter how clever or original it might be. I guess this joke just isn't my cup of tea. I appreciate the effort, but it's hard for me to look past the groan-worthy nature of the punchline.
------------------------------------------------------------

🔹 Judge 

In [18]:
min = score_list[0]
max = score_list[4]
for i in score_list:
    if i!=None and i<min:
        min = i
    elif i != None and i>max:
        max = i
print(min)
print(max)

2
8


In [20]:
print("Top Joke from the scoring of our respected judges is...\n\n")
for score_and_joke_dicts in score_and_joke_list:
    if (score_and_joke_dicts['score'] == max):
        print(score_and_joke_dicts['joke'])

Top Joke from the scoring of our respected judges is...
Why did the penguin bring a ladder to the bar? Because he wanted to reach the high-smС
What do you call a penguin that love the trapeze? A frozen tough of the track


In [2]:
import re

def extract_final_score(verdict_text):
    match = re.search(r'(\d+)\s+out of\s+10', verdict_text)
    if match:
        return int(match.group(1))
    else:
        return None

score = extract_final_score(verdict)
print(f"🎯 Final score: {score}")

🎯 Final score: 7
